In [9]:
import os
import pandas as pd
import leafmap

In [10]:
df = pd.read_csv('532572_2456265_misc.csv')
df.head()

,都道府県コード又は市区町村コード,NO,都道府県名,データ名称,データ概要,データ形式,分類,URL,ライセンス,掲載ページ名,掲載ページURL,データ提供所属,備考
0,230006,1,愛知県,ひったくり（CSV：4KB）,令和4年中の犯罪発生情報,csv,くらし・安全,https://www.pref.aichi.jp/police/anzen/toukei/...,CC BY,愛知県警察犯罪オープンデータサイト,https://www.pref.aichi.jp/police/anzen/toukei/...,愛知県警察本部,NaN
1,230006,2,愛知県,車上ねらい（CSV：162KB）,令和4年中の犯罪発生情報,csv,くらし・安全,https://www.pref.aichi.jp/police/anzen/toukei/...,CC BY,愛知県警察犯罪オープンデータサイト,https://www.pref.aichi.jp/police/anzen/toukei/...,愛知県警察本部,NaN
2,230006,3,愛知県,部品ねらい（CSV：133KB）,令和4年中の犯罪発生情報,csv,くらし・安全,https://www.pref.aichi.jp/police/anzen/toukei/...,CC BY,愛知県警察犯罪オープンデータサイト,https://www.pref.aichi.jp/police/anzen/toukei/...,愛知県警察本部,NaN
3,230006,4,愛知県,自動販売機ねらい（CSV：31KB）,令和4年中の犯罪発生情報,csv,くらし・安全,https://www.pref.aichi.jp/police/anzen/toukei/...,CC BY,愛知県警察犯罪オープンデータサイト,https://www.pref.aichi.jp/police/anzen/toukei/...,愛知県警察本部,NaN
4,230006,5,愛知県,自動車盗（CSV：93KB）,令和4年中の犯罪発生情報,csv,くらし・安全,https://www.pref.aichi.jp/police/anzen/toukei/...,CC BY,愛知県警察犯罪オープンデータサイト,https://www.pref.aichi.jp/police/anzen/toukei/...,愛知県警察本部,NaN


In [11]:
# 都道府県コード又は市区町村コード,NO,都道府県名,データ名称,データ概要,データ形式,分類,URL,ライセンス,掲載ページ名,掲載ページURL,データ提供所属,備考

# データ形式 が csv かつ、データ概要が "景観資源" のものを抽出
df_filtered = df[(df['データ形式'] == 'csv') & (df['データ概要'].str.contains('景観資源'))]

# URL からデータを取得して、"LandscapeResources" ディレクトリに保存

output_dir = 'LandscapeResources'
os.makedirs(output_dir, exist_ok=True)

import requests
from io import BytesIO
import zipfile

# ファイル名のサニタイズ
def sanitize_filename(s):
    return "".join(c if (c.isalnum() or c in (' ','.','_','-')) else '_' for c in str(s)).strip()

# バイトデータから pandas.read_csv を複数エンコーディングで試行
def try_read_csv_from_bytes(b):
    encodings = ['utf-8', 'cp932', 'shift_jis', 'euc_jp', None]
    last_exc = None
    for enc in encodings:
        try:
            if enc is None:
                return pd.read_csv(BytesIO(b))
            else:
                return pd.read_csv(BytesIO(b), encoding=enc)
        except Exception as e:
            last_exc = e
    raise last_exc

for index, row in df_filtered.iterrows():
    url = row['URL']
    prefecture = row.get('都道府県名', 'unknown')
    name = row.get('データ名称', 'data')
    print(f"Processing {url} for {prefecture} - {name}")
    if pd.notna(url) and str(url).strip():
        try:
            resp = requests.get(url, timeout=30)
            resp.raise_for_status()
            content = resp.content

            # ZIPファイル判定: URL末尾、もしくはバイナリを確認
            is_zip = False
            try:
                is_zip = zipfile.is_zipfile(BytesIO(content))
            except Exception:
                is_zip = False

            if is_zip:
                with zipfile.ZipFile(BytesIO(content)) as z:
                    csv_names = [n for n in z.namelist() if n.lower().endswith('.csv')]
                    if not csv_names:
                        raise ValueError('ZIP の中に CSV が見つかりませんでした')
                    with z.open(csv_names[0]) as f:
                        b = f.read()
                        data = try_read_csv_from_bytes(b)
                        csv_part = os.path.splitext(os.path.basename(csv_names[0]))[0]
            else:
                data = try_read_csv_from_bytes(content)
                csv_part = name

            file_name = f"{prefecture}_{csv_part}.csv"
            file_name = sanitize_filename(file_name)
            out_path = os.path.join(output_dir, file_name)
            data.to_csv(out_path, index=False)
            print(f"Saved to {out_path}")
        except Exception as e:
            print(f"Error processing {url}: {e}")
    else:
        print(f"No URL for row {index}")

Processing https://profile.maps.pref.aichi.jp/map/files/opendata/c200538/c200538.csv for 愛知県 - 名古屋市（csv：2.7KB）
Saved to LandscapeResources/愛知県_名古屋市_csv_2.7KB_.csv
Processing https://profile.maps.pref.aichi.jp/map/files/opendata/c200539/c200539.csv for 愛知県 - 豊橋市（csv：1.0KB）
Saved to LandscapeResources/愛知県_豊橋市_csv_1.0KB_.csv
Processing https://profile.maps.pref.aichi.jp/map/files/opendata/c200540/c200540.csv for 愛知県 - 岡崎市（csv：1.2KB）
Saved to LandscapeResources/愛知県_名古屋市_csv_2.7KB_.csv
Processing https://profile.maps.pref.aichi.jp/map/files/opendata/c200539/c200539.csv for 愛知県 - 豊橋市（csv：1.0KB）
Saved to LandscapeResources/愛知県_豊橋市_csv_1.0KB_.csv
Processing https://profile.maps.pref.aichi.jp/map/files/opendata/c200540/c200540.csv for 愛知県 - 岡崎市（csv：1.2KB）
Saved to LandscapeResources/愛知県_岡崎市_csv_1.2KB_.csv
Processing https://profile.maps.pref.aichi.jp/map/files/opendata/c200541/c200541.csv for 愛知県 - 一宮市（csv：2.5KB）
Saved to LandscapeResources/愛知県_一宮市_csv_2.5KB_.csv
Processing https://profile.maps

In [12]:
# 'LandscapeResources' の中のcsvファイルを全て統合する
# head フィーチャID,形状(WKT),タイトル,説明,スタイルID,期間開始日時,期間終了日時,作成者,作成日時,更新者,更新日時 
# head に 県,市区町村名,を追加する

import glob

all_files = glob.glob(os.path.join(output_dir, '*.csv'))
combined_data = []

for file in all_files:
    try:
        data = pd.read_csv(file)
        if '都道府県名' not in data.columns:
            data['都道府県名'] = os.path.basename(file).split('_')[0]
        if '市区町村名' not in data.columns:
            data['市区町村名'] = os.path.basename(file).split('_')[1] if len(os.path.basename(file).split('_')) > 1 else 'unknown'
        combined_data.append(data)
    except Exception as e:
        print(f"Error reading {file}: {e}")

if combined_data:
    combined_df = pd.concat(combined_data, ignore_index=True)
    combined_df = combined_df[['都道府県名', '市区町村名'] + [col for col in combined_df.columns if col not in ['都道府県名', '市区町村名']]]
    combined_df.to_csv('combined_landscape_resources.csv', index=False)
    print("Combined data saved to 'combined_landscape_resources.csv'")

Combined data saved to 'combined_landscape_resources.csv'


In [13]:
# ポイントを可視化する
from shapely.wkt import loads

def extract_coordinates(wkt):
    try:
        point = loads(wkt)
        return point.x, point.y
    except Exception as e:
        print(f"Error parsing WKT: {wkt}, {e}")
        return None, None

combined_df['x'], combined_df['y'] = zip(*combined_df['形状(WKT)'].apply(extract_coordinates))

m = leafmap.Map(center=[35.0, 136.9], zoom=8)
m.add_points_from_xy(
    combined_df,
    x="x",
    y="y",
    color="スタイルID",
    popup=["タイトル", "説明"]
)

m

Map(center=[35.0, 136.9], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_ou…

In [15]:
# points の情報を json として保存
"""
{
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "geometry": {
        "type": "Point",
        "coordinates": [136.9, 35.0]
      },
      "properties": {
        "スタイルID": 1,
        "タイトル": "ポイント1",
        "説明": "説明1"
      }
    }
  ]
}
"""

import json

features = []
for _, row in combined_df.iterrows():
    if pd.notna(row['x']) and pd.notna(row['y']):
        feature = {
            "type": "Feature",
            "geometry": {
                "type": "Point",
                "coordinates": [row['x'], row['y']]
            },
            "properties": {
                "スタイルID": row['スタイルID'],
                "タイトル": row['タイトル'],
                "説明": row['説明']
            }
        }
        features.append(feature)

geojson = {
    "type": "FeatureCollection",
    "features": features
}

with open('landscape_resources.geojson', 'w', encoding='utf-8') as f:
    json.dump(geojson, f, ensure_ascii=False, indent=2)